<a href="https://colab.research.google.com/github/danielle-l23/hello-world/blob/main/The_Medical_Home.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Group Members:** 9079414778, 9071987730, 9058163032

First, we'll install some packages necessary to display our tables appropriately.

In [ ]:
library("epiDisplay")

library("questionr")

Now we can read in our data.

In [ ]:
load('H171.RData')

For the study, we are looking exclusively at children who have had office visit for health care. Therefore, we are removing people over 17 years old, and people who have 0 office-based health care visits. 

In [ ]:
#Remove all people over 17yrs old
dat <- H171[which(H171$AGE14X < 18 & H171$AGE14X > 0), ]

#Remove observations with missing value for age variable
dat$AGE14X[dat$AGE14X == -1] <- NA

summary(dat$AGE14X)

#Remove all missing values and people who have 0 office-based health care visits
dat <- dat[which(dat$CHAPPT42 > 0), ]

summary(dat$CHAPPT42)

## Missing Data

Next we will examine the missing data in the MEPS 2014 variables we will be using.

In [ ]:
#Variables for Table 1/Table 3
table1_dat <- dat[, c('HAVEUS42', 'PROVTY42', 'OFFHOU42', 'AFTHOU42', 'PHNREG42', 'CHLIST42', 'CHPRTM42', 'TREATM42', 'RESPCT42', 'DECIDE42', 'EXPLOP42', 'CHEXPL42', 'CHSPEC42', 'CHEYRE42', 'CHILCR42', 'CHILWW42', 'CHRTCR42', 'CHRTWW42', 'CHNDCR42', 'CHENEC42', 'CHRESP42')]

#Variables for Table 2
demo_dat <- dat[,c('AGE14X', 'INSCOV14', 'sex', 'POVLEV14', 'racethx', 'RTHLTH53', 'REGION14')]

#Combine data frames
comp_dat <- cbind(demo_dat, table1_dat)

summary(comp_dat)

#Remove observations in data frame coded as a missing value by MEPS
comp_dat[comp_dat < 0] <- NA

In [ ]:
#Produce chart showing count of missing values and percentage missing
#for each variable
freq.na(comp_dat)

## Table 2

Now, we'll target the variables for Table 2, which focuses on the demographics of all children in the MEPS dataset that have had at least 1 office-based healthcare visit in the year prior to data collection. 

'AGE14X' is a numerical variable of age collected in 2014. 'INSCOV14' gives a breakdown of basic insurance types by patient, also collected in 2014. 'sex' gives binary gender information (the same in all MEPS versions). 'POVLEV14' indicates family income of participant as a % of the federal poverty line (these data were collected in 2014). 'racethx' indicates race and ethnicity, chosen specifically because of its question regarding Hispanic ethnicity. 'RTHLTH53' indicates perceived health status - in this case it would be parent-reported perceived health status of the child. Note that 'RTHLTH53' did not have an available version from 2014, so the version from 5/3 was chosen in an effort to use the most recent data possible. 'REGION14' gives information on regional residence of the participant (data collected in 2014).


In [ ]:
#Create new categorical variables to match categories in the paper for Table 2
demo_dat <- within(demo_dat, {
        Gender <- NA
        Gender[sex == 1] <- "Male"
        Gender[sex == 2] <- "Female"
        Age <- NA
        Age[AGE14X >= 0 & AGE14X <= 5] <- "0-5 Years"
        Age[AGE14X > 5 & AGE14X <= 11] <- "6-11 Years"
        Age[AGE14X > 11 & AGE14X <= 17] <- "12-17 Years"
        Race_ethnicity <- NA
        Race_ethnicity[racethx == 1] <- "Hispanic"
        Race_ethnicity[racethx == 2] <- "White, non-Hispanic"
        Race_ethnicity[racethx == 3] <- "Black, non-Hispanic"
        Race_ethnicity[racethx == 4 | racethx == 5] <- "Other, non-Hispanic"
        Insurance <- NA
        Insurance[INSCOV14 == 1] <- "Private"
        Insurance[INSCOV14 == 2] <- "Public"
        Insurance[INSCOV14 == 3] <- "Uninsured Full Year"
        FamilyIncome <- NA
        FamilyIncome[POVLEV14 < 100] <- "<100% FPL"
        FamilyIncome[POVLEV14 >= 100 & POVLEV14 < 200] <- "100%–199% FPL"
        FamilyIncome[POVLEV14 >= 200 & POVLEV14 < 400] <- "200%–399% FPL"
        FamilyIncome[POVLEV14 >= 400] <- ">=400% FPL"
        ChildHealth <- NA
        ChildHealth[RTHLTH53 == 1 | RTHLTH53 == 2] <- "Excellent/very good"
        ChildHealth[RTHLTH53 == 3] <- "Good"
        ChildHealth[RTHLTH53 == 4 | RTHLTH53 == 5] <- "Fair/poor"
        GeographicRegion <- NA
        GeographicRegion[REGION14 == 1] <- "Northeast"
        GeographicRegion[REGION14 == 2] <- "Midwest"
        GeographicRegion[REGION14 == 3] <- "South"
        GeographicRegion[REGION14 == 4] <- "West"
               })

In [ ]:
#Convert new categorical variables to factors
factor_vars <- c("Age", "Gender", "Race_ethnicity", "Insurance", "FamilyIncome", "ChildHealth", "GeographicRegion")

demo_dat[factor_vars] <- lapply(demo_dat[factor_vars], factor)

In [ ]:
#Remove original variables
demo_dat[,1:7] <- list(NULL)

summary(demo_dat)
str(demo_dat)

Now we can find the percentages for Table 2

In [ ]:
#Age Breakdown
tab1(demo_dat$Age, sort.group = "decreasing")

#Gender Breakdown
tab1(demo_dat$Gender, sort.group = "decreasing")

#Race/ethnicity Breakdown
tab1(demo_dat$Race_ethnicity, sort.group = "decreasing", cex = 1, cex.names = .75)

#Insurance Breakdown
tab1(demo_dat$Insurance, sort.group = "decreasing")

#Family income Breakdown
tab1(demo_dat$FamilyIncome, sort.group = "decreasing", cex = 1, cex.names = .75)

#Parent report HS Breakdown
tab1(demo_dat$ChildHealth, sort.group = "decreasing")

#Region Breakdown
tab1(demo_dat$GeographicRegion, sort.group = "decreasing")

## Table 1 / Table 3
The first step to create Table 3 is to code the survey items from Table 1 so that they can be used to generate Table 3.

In [ ]:
#Create new categorical variables to match survey items in Table 1
table1_dat <- within(table1_dat, {
      Offhours <- NA
      Offhours[OFFHOU42 == 1] <- 100
      Offhours[OFFHOU42 == 2 | OFFHOU42 == -8] <- 0
      Diff_Aft <- NA
      Diff_Aft[AFTHOU42 == 1] <- 0
      Diff_Aft[AFTHOU42 == 2] <- 25
      Diff_Aft[AFTHOU42 == 3] <- 75
      Diff_Aft[AFTHOU42 == 4] <- 100
      Phone <- NA
      Phone[PHNREG42 == 1] <- 0
      Phone[PHNREG42 == 2] <- 25
      Phone[PHNREG42 == 3] <- 75
      Phone[PHNREG42 == 4] <- 100
      Listened <- NA
      Listened[CHLIST42 == 1] <- 0
      Listened[CHLIST42 == 2] <- 25
      Listened[CHLIST42 == 3] <- 75
      Listened[CHLIST42 == 4] <- 100
      EnoughTime <- NA
      EnoughTime[CHPRTM42 == 1] <- 0
      EnoughTime[CHPRTM42 == 2] <- 25
      EnoughTime[CHPRTM42 == 3] <- 75
      EnoughTime[CHPRTM42 == 4] <- 100
      Advice <- NA
      Advice[TREATM42 == 1] <- 100
      Advice[TREATM42 == 2 | TREATM42 == -8] <- 0
      Respect <- NA
      Respect[RESPCT42 == 1] <- 0
      Respect[RESPCT42 == 2] <- 25
      Respect[RESPCT42 == 3] <- 75
      Respect[RESPCT42 == 4] <- 100
      Decide <- NA
      Decide[DECIDE42 == 1] <- 0
      Decide[DECIDE42 == 2] <- 25
      Decide[DECIDE42 == 3] <- 75
      Decide[DECIDE42 == 4] <- 100
      ExplainOpt <- NA
      ExplainOpt[EXPLOP42 == 1] <- 100
      ExplainOpt[EXPLOP42 == 2 | EXPLOP42 == -8] <- 0
      Understand <- NA
      Understand[CHEXPL42 == 1] <- 0
      Understand[CHEXPL42 == 2] <- 25
      Understand[CHEXPL42 == 3] <- 75
      Understand[CHEXPL42 == 4] <- 100
      Access_Spec <- NA
      Access_Spec[CHEYRE42 == 1 | CHEYRE42 == -8] <- 100
      Access_Spec[CHEYRE42 == 2] <- 50
      Access_Spec[CHEYRE42 == 3 | CHEYRE42 == 4] <- 0
      CareNeeded <- NA
      CareNeeded[CHILWW42 == 1 | CHILWW42 == -8] <- 0
      CareNeeded[CHILWW42 == 2] <- 25
      CareNeeded[CHILWW42 == 3] <- 75
      CareNeeded[CHILWW42 == 4] <- 100
      RoutineCare <- NA
      RoutineCare[CHRTWW42 == 1] <- 0
      RoutineCare[CHRTWW42 == 2] <- 25
      RoutineCare[CHRTWW42 == 3] <- 75
      RoutineCare[CHRTWW42 == 4 | CHRTWW42 == -8] <- 100
      NecessaryCare <- NA
      NecessaryCare[CHENEC42 == 1] <- 0
      NecessaryCare[CHENEC42 == 2] <- 25
      NecessaryCare[CHENEC42 == 3] <- 75
      NecessaryCare[CHENEC42 == 4 | CHENEC42 == -8] <- 100
      RespPat <- NA
      RespPat[CHRESP42 == 1] <- 0
      RespPat[CHRESP42 == 2] <- 25
      RespPat[CHRESP42 == 3] <- 75
      RespPat[CHRESP42 == 4] <- 100
})

In [ ]:
#Display first 6 rows of the new categorical variables
head(table1_dat[,c(22:36)])

In [ ]:
#Create additional categorical variables to match survey items in Table 1 
table1_new <- within(table1_dat, {
  UsualSource <- NA
  UsualSource[HAVEUS42 == 1] <- "Yes"
  UsualSource[HAVEUS42 == 2] <- "No"
  ProvType <- NA
  ProvType[PROVTY42 == 1] <- "Facility"
  ProvType[PROVTY42 == 2 & 3] <- "Person"
  Offhours <- NA
  Offhours[OFFHOU42 == 1] <- 100
  Offhours[OFFHOU42 == 2] <- 0
  Diff_Aft <- NA
  Diff_Aft[AFTHOU42 == 1] <- 0
  Diff_Aft[AFTHOU42 == 2] <- 25
  Diff_Aft[AFTHOU42 == 3] <- 75
  Diff_Aft[AFTHOU42 == 4] <- 100
  Phone <- NA
  Phone[PHNREG42 == 1] <- 0
  Phone[PHNREG42 == 2] <- 25
  Phone[PHNREG42 == 3] <- 75
  Phone[PHNREG42 == 4] <- 100
  Listened <- NA
  Listened[CHLIST42 == 1] <- 0
  Listened[CHLIST42 == 2] <- 25
  Listened[CHLIST42 == 3] <- 75
  Listened[CHLIST42 == 4] <- 100
  EnoughTime <- NA
  EnoughTime[CHPRTM42 == 1] <- 0
  EnoughTime[CHPRTM42 == 2] <- 25
  EnoughTime[CHPRTM42 == 3] <- 75
  EnoughTime[CHPRTM42 == 4] <- 100
  Advice <- NA
  Advice[TREATM42 == 1] <- 100
  Advice[TREATM42 == 2] <- 0
  Respect <- NA
  Respect[RESPCT42 == 1] <- 0
  Respect[RESPCT42 == 2] <- 25
  Respect[RESPCT42 == 3] <- 75
  Respect[RESPCT42 == 4] <- 100
  Decide <- NA
  Decide[DECIDE42 == 1] <- 0
  Decide[DECIDE42 == 2] <- 25
  Decide[DECIDE42 == 3] <- 75
  Decide[DECIDE42 == 4] <- 100
  ExplainOpt <- NA
  ExplainOpt[EXPLOP42 == 1] <- 100
  ExplainOpt[EXPLOP42 == 2] <- 0
  Understand <- NA
  Understand[CHEXPL42 == 1] <- 0
  Understand[CHEXPL42 == 2] <- 25
  Understand[CHEXPL42 == 3] <- 75
  Understand[CHEXPL42 == 4] <- 100
  Specialist <- NA
  Specialist[CHSPEC42 == 1] <- 1
  Specialist[CHSPEC42 == 2] <- 0
  Access_Spec <- NA
  Access_Spec[CHEYRE42 == 1] <- 100
  Access_Spec[CHEYRE42 == 2] <- 50
  Access_Spec[CHEYRE42 == 3 | CHEYRE42 == 4] <- 0
  Immediate <- NA
  Immediate[CHILCR42 == 1] <- 100
  Immediate[CHILCR42 == 2] <- 0
  CareNeeded <- NA
  CareNeeded[CHILWW42 == 1] <- 0
  CareNeeded[CHILWW42 == 2] <- 25
  CareNeeded[CHILWW42 == 3] <- 75
  CareNeeded[CHILWW42 == 4] <- 100
  Routine <- NA
  Routine[CHRTCR42 == 1] <- 100
  Routine[CHRTCR42 == 2] <- 0
  RoutineCare <- NA
  RoutineCare[CHRTWW42 == 1] <- 0
  RoutineCare[CHRTWW42 == 2] <- 25
  RoutineCare[CHRTWW42 == 3] <- 75
  RoutineCare[CHRTWW42 == 4] <- 100
  Believe <- NA
  Believe[CHNDCR42 == 1] <- 100
  Believe[CHNDCR42 == 2] <- 100
  NecessaryCare <- NA
  NecessaryCare[CHENEC42 == 1] <- 0
  NecessaryCare[CHENEC42 == 2] <- 25
  NecessaryCare[CHENEC42 == 3] <- 75
  NecessaryCare[CHENEC42 == 4] <- 100
  RespPat <- NA
  RespPat[CHRESP42 == 1] <- 0
  RespPat[CHRESP42 == 2] <- 25
  RespPat[CHRESP42 == 3] <- 75
  RespPat[CHRESP42 == 4] <- 100
})

In [ ]:
#add to original working table for ease
table1_new$access <- rowMeans(table1_new[, c('Offhours', 'Diff_Aft', 'Phone')], na.rm = True)

table1_new$famcenter <- rowMeans(table1_new[, c('Listened', 'EnoughTime','Advice',
                                                'Respect','Decide','ExplainOpt',
                                                'Understand')], na.rm = True)

table1_new$comprehensive <- rowMeans(table1_new[, c('Specialist','Access_Spec',
                                                    'Immediate','CareNeeded',
                                                    'Routine','RoutineCare',
                                                    'Believe','NecessaryCare')], na.rm = True)

In [ ]:
#build the binary indicator for each category for medical homes
table1_new <- within(table1_new, {
  Accessible <- NA
  Accessible[access >= 75] <- 1
  Accessible[access < 75] <- 0
})
table1_new <- within(table1_new, {
  FamilyCentered <- NA
  FamilyCentered[famcenter >= 75] <- 1
  FamilyCentered[famcenter < 75] <- 0
})
table1_new <- within(table1_new, {
  Comprehensive <- NA
  Comprehensive[comprehensive >= 75] <- 1
})
table1_new <- within(table1_new, {
  Compassionate <- NA
  Compassionate[RespPat >= 75] <- 1
  Compassionate[RespPat < 75] <- 0
})
table1_new$MHScores <- rowSums(table1_new[ , c('Accessible','FamilyCentered',
                                               'Comprehensive','Compassionate')])
table1_new <- within(table1_new, {
  MedicalHome <- NA
  MedicalHome[MHScores == 4] <- "Yes MH"
  MedicalHome[MHScores < 3] <- "No MH"
})
table(table1_new$MedicalHome) #note ~122 in MH and 34 not, as of our current version

In [ ]:
#start creating table 3 using "table1_new"
table3noprop <- table1_new[ , c('MedicalHome','UsualSource','ProvType','Accessible',
                                'FamilyCentered','Comprehensive','Compassionate')]

head(table3noprop) 

Note all the NAs in the Medical Home and Comprehensive category. This is mainly due to aggregating the data when there were multiple missings. If we were to do this again, we would need to either remove variables that are portions of the questions we matched that have large missing values (as even just one missing will lead to an NA) or we find a way to solve for missing values. This leads to ~450 records left for comprehensive and ~150 for medical homes. 

In [ ]:
#Display tables for each element of medical homes listed in Table 3
table(table3noprop$ProvType)

table(table3noprop$Accessible)

table(table3noprop$Accessible,table3noprop$ProvType)

table(table3noprop$FamilyCentered)
table(table3noprop$Comprehensive)
table(table3noprop$Compassionate)
table(table3noprop$MedicalHome)
table(table3noprop$MedicalHome,table3noprop$ProvType)

In [ ]:
###############################
## Didn't use in Deliverable 1
###############################
for (row in range(length(table1_new))) {
  if (table1_new[row, 'Specialist'] == 1){
    if (table1_new[row, 'Immediate'] == 1) {
      if (table1_new[row, 'Routine'] == 1) {
        if (table1_new[row, 'Believe'] == 1){
          table1_new$comprehensive <- rowMeans(table1_new[, c('Access_Spec','CareNeeded',
                                                    'RoutineCare','NecessaryCare')], na.rm = TRUE)}
        
        else {
          table1_new$comprehensive <- rowMeans(table1_new[, c('Access_Spec','CareNeeded',
                                                    'RoutineCare')])}
      }                                             
      else {
        if (table1_new[row, 'Believe'] == 1){
          table1_new$comprehensive <- rowMeans(table1_new[, c('Access_Spec', 'CareNeeded', 'Believe')])}
        else {
          table1_new$comprehensive <- rowMeans(table1_new[, c('Access_Spec', 'CareNeeded')])
        } 
      }
        }
    else {
      if (table1_new[row, 'Routine'] == 1){
        if (table1_new[row, 'Believe'] == 1) {
          table1_new$comprehensive <- rowMeans(table1_new[, c('Access_Spec', 'RoutineCare', 'NecessaryCare'), na.rm = TRUE])
        }
        else {
          table1_new$comprehensive <- rowMeans(table1_new[, c('Access_Spec', 'RoutineCare')])
        }
      }
      else {
        if (table1_new[row, 'Believe'] == 1) {
          table1_new$comprehensive <- rowMeans(table1_new[, c('Access_Spec', 'NecessaryCare')], na.rm = TRUE)}
        else {
          table1_new$comprehensive <- mean(table1_new[, 'Access_Spec'])
        }
        }
      }
    }    
  else {
    if (table1_new[row, 'Immediate'] == 1) {
      if (table1_new[row, 'Routine'] == 1) {
        if (table1_new[row, 'Believe'] == 1){
          table1_new$comprehensive <- rowMeans(table1_new[, c('CareNeeded',
                                                    'RoutineCare','NecessaryCare')], na.rm = TRUE)}
        else {
          table1_new$comprehensive <- rowMeans(table1_new[, c('CareNeeded', 'RoutineCare')], na.rm = TRUE)
        }
      }
      else {
        if (table1_new[row, 'Believe'] == 1){
          table1_new$comprehensive <- rowMeans(table1_new[, c('CareNeeded', 'NecessaryCare')], na.rm = TRUE)}
        else {
          table1_new$comprehensive <- mean(table1_new[, 'CareNeeded'])
        }
      }
    }
    else {
      if (table1_new[row, 'Routine'] == 1) {
        if (table1_new[row, 'Believe'] == 1){
          table1_new$comprehensive <- rowMeans(table1_new[, c('RoutineCare', 'NecessaryCare')], na.rm = TRUE)}
        else {
          table1_new$comprehensive <- mean(table1_new[, 'RoutineCare'])
        }
      }
      else {
        if (table1_new[row, 'Believe'] == 1) {
          table1_new$comprehensive <- mean(table1_new[, 'NecessaryCare'], na.rm = TRUE)
        }
        else{
          table1_new$comprehensive <- 0
        }
      }
    }
  }
  
}
